# Adding a Custom Pipeline Component on Spacy 

In this document, we modify Spacy's pipeline, creating a custom component that normalizes word spelling. This is an example intended to show ways to improve the pipeline for historical texts.


In [32]:
#Import modules used in this notebook

import spacy
from spacy.lang.es import Spanish 
from spacy import displacy

import json
from copy import deepcopy

## Understanding the Pipeline 

<div>
<img src=https://spacy.io/pipeline-7a14d4edd18f3edfee8f34393bff2992.svg  width="700"/>
</div>
When analyzing a text, Spacy takes a string and passes it through a chain of processes. First, it tokenizes the text, separating it into the individual words contained in the document. Then, it applies other processes (by default, the tagger, the parser, and the named entity recognizer) to your document. These components are independent and can be disabled or dropped, depending on your NLP needs. New, custom components can be added (and the tokenizer can also be modified).

We are interested in improving the performance of the NER component, which translates each token into a numerical vector calculated from each token's prefix, suffix, norm and shape attributes. In this notebook, we add a custom component that takes historical texts, with its multiple variations in spelling, and normalizes spelling of the norm attribute to bring it closer to modern usage. Because the Spacy model used was trained on texts written in modern Spanish, this should improve performance.

In [33]:
#First, load the model
nlp= spacy.load('es_core_news_md')

In [34]:
#Create model copy to modify (allows us to compare results before and after modification)
nlp1= deepcopy(nlp)

## Modifying the Pipeline

We have created a dictionary of early modern Spanish words, culled from a subset of documents and edited on OpenRefine, with their modern spanish equivalents. These are available in a file saved separately, which we call here.

In [35]:
# read from JSON file
with open('normalizeddict.json', 'r', encoding='utf-8') as fp2:
    normalizeddict = json.load(fp2)

We then define the python function that will run within the pipeline:

In [10]:
#Define and add pipeline component that updates .norm attribute

def add_custom_norms(doc):
    for token in doc:
        if token.text in normalizeddict:
            token.norm_ = normalizeddict[token.text]
    return doc

And add the component to the pipeline.

In [36]:
#Add component to the pipeline

nlp1.add_pipe(add_custom_norms, first=True)

## Viewing pipeline component results

In [38]:
#Processing a text with the unmodified and the modified pipelines.

doc=nlp("juan baptista bazquez escultor vzo en san pedro como principal e diego de çamora pintor de ymagineria como su fiador otorgamos que somos concertados con juan rruiz de porras vzo de la ciudad de Ronda en nonbre de los texedores de paños e lienços de Ronda en tal manera que nos obligamos de hazer de talla e pintura un san marcos que tenga sin la peana bara e media de alto e la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de yr la ynsinia que es un toro del tamaño que le conuenga. el qual santo a de yr vestido como de apostol y a de ser de madera de pino de sigura a de yr hueco para que se pueda lleuar en prosesion y todo el y la peana a de yr dorado y estofado todos los uestidos conforme e de la misma manera questa eftofado una ymagen de señor san josehpe questa en la casa profesa de la conpañia de jesus en la capilla mayor en un altar y nos obligamos a hazer una caxa de madera de pino en que vaya metido para lleuar de aqui a Ronda y mas nos obligamos a hazer unas pariguelas para sacallo en prosesion las tablas de pino y los braços de borne jaspeadas de verde a el olio y el santo a de llevar en la mano ysquierda un libro dorado y en la mano derecha una pluma y en la cabeça una diadema dorada todo ello de buena obra y nos obligamos de lo dar acabado de aqui a el dia de san juan bautista del mes de junio deste año por precio todo ello de quarenta ducados y rrecibo para en quenta 17 ducados.")
doc1=nlp1("juan baptista bazquez escultor vzo en san pedro como principal e diego de çamora pintor de ymagineria como su fiador otorgamos que somos concertados con juan rruiz de porras vzo de la ciudad de Ronda en nonbre de los texedores de paños e lienços de Ronda en tal manera que nos obligamos de hazer de talla e pintura un san marcos que tenga sin la peana bara e media de alto e la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de yr la ynsinia que es un toro del tamaño que le conuenga. el qual santo a de yr vestido como de apostol y a de ser de madera de pino de sigura a de yr hueco para que se pueda lleuar en prosesion y todo el y la peana a de yr dorado y estofado todos los uestidos conforme e de la misma manera questa eftofado una ymagen de señor san josehpe questa en la casa profesa de la conpañia de jesus en la capilla mayor en un altar y nos obligamos a hazer una caxa de madera de pino en que vaya metido para lleuar de aqui a Ronda y mas nos obligamos a hazer unas pariguelas para sacallo en prosesion las tablas de pino y los braços de borne jaspeadas de verde a el olio y el santo a de llevar en la mano ysquierda un libro dorado y en la mano derecha una pluma y en la cabeça una diadema dorada todo ello de buena obra y nos obligamos de lo dar acabado de aqui a el dia de san juan bautista del mes de junio deste año por precio todo ello de quarenta ducados y rrecibo para en quenta 17 ducados.")

In [26]:
#Displaying unmodified text
for token in doc1:
    print(token.text,end=' ')

juan baptista bazquez escultor vzo en san pedro como principal e diego de çamora pintor de ymagineria como su fiador otorgamos que somos concertados con juan rruiz de porras vzo de la ciudad de Ronda en nonbre de los texedores de paños e lienços de Ronda en tal manera que nos obligamos de hazer de talla e pintura un san marcos que tenga sin la peana bara e media de alto e la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de yr la ynsinia que es un toro del tamaño que le conuenga . el qual santo a de yr vestido como de apostol y a de ser de madera de pino de sigura a de yr hueco para que se pueda lleuar en prosesion y todo el y la peana a de yr dorado y estofado todos los uestidos conforme e de la misma manera questa eftofado una ymagen de señor san josehpe questa en la casa profesa de la conpañia de jesus en la capilla mayor en un altar y nos obligamos a hazer una caxa de madera de pino en que vaya metido para lleuar de aqui a Ronda y mas nos obligamos a haze

In [41]:
#Displaying normalized text
for token in doc1:
    print(token.norm_, end=' ')

Juan Bautista bazquez escultor vzo en san Pedro como principal y Diego de Zamora pintor de imagineria como su fiador otorgamos que somos concertados con Juan rruiz de Porras vzo de la ciudad de ronda en nombre de los texedores de paños y lienzos de ronda en tal manera que nos obligamos de hacer de talla y pintura un san Marcos que tenga sin la peana vara y media de alto y la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de ir la insignia que es un toro del tamaño que le convenga . el cual santo a de ir vestido como de apostol y a de ser de madera de pino de sigura a de ir hueco para que se pueda llevar en prosesion y todo el y la peana a de ir dorado y estofado todos los uestidos conforme y de la misma manera que esta eftofado una imagen de señor san josehpe que esta en la casa profesa de la compañia de Jesus en la capilla mayor en un altar y nos obligamos a hacer una caja de madera de pino en que vaya metido para llevar de aqui a ronda y mas nos obligamos a

In [39]:
#Displaying recognized entities in this document, with unmodified pipeline
displacy.render(doc, style='ent', jupyter= True)

In [40]:
#Displaying recognized entities in the document, with modified pipeline
displacy.render(doc1, style='ent', jupyter= True)

As can be seen, there are some differences in the way the NER pipeline component identifies entities in the document, though whether there it is unclear whether this is an improvement. Thorough performance testing indicates that there is not, so we will also explore ways to instead modify the tokenizer to take into account spelling differences. 